In [4]:
from preprocess import Preprocess
import pandas as pd
import numpy as np
import traceback

In [9]:
    data_path = '../datas/spark.csv'
    df = pd.read_csv(open(data_path, 'rU'))
    df['Duplicate_null'] = df['Duplicated_issue'].apply(lambda x : pd.isnull(x))
    prep = Preprocess()
    df['Desc_list'] = df['Title'].apply(lambda x : prep.stem_and_stop_removal(x))
    # Positive samples
    df_data = df[df['Duplicate_null'] == False]


    df_field = df_data[['Issue_id', 'Desc_list', 'Duplicated_issue', 'Resolution']]
    df_field['dup_list'] = df_field['Duplicated_issue'].apply(lambda x: x.split(';'))
    Dup_list = []
    for i,r in df_field.iterrows():
        for dup in r['dup_list']:
            if int(r['Issue_id'].split('-')[1]) < int(dup.split('-')[1]):
                if dup.startswith('SPARK'):
                    Dup_list.append([r['Issue_id'], dup, r['Resolution']])
    df_pairs_pos = pd.DataFrame(Dup_list, columns = ['Issue_id_1', 'Issue_id_2', 'Resolution'])

    # Negative samples
    neg_dup_list = []
    cnt = 0
    for i,r in df.iterrows():
        if r['Duplicate_null'] == True:
            j = 1
            try:
                while not df.ix[i+j]['Issue_id'].startswith('SPARK'):
                    j += 1
                neg_dup_list.append([r['Issue_id'], df.ix[i+j]['Issue_id'], r['Resolution']])
                cnt += 1
            except:
                print traceback.print_exc() 
            
        if cnt > len(Dup_list):
            break

    df_pairs_neg = pd.DataFrame(neg_dup_list, columns = ['Issue_id_1', 'Issue_id_2', 'Resolution'])


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
df_pairs_neg['Title_1'] = df_pairs_neg['Issue_id_1'].apply(lambda x: list(df[df['Issue_id'] == x]['Desc_list'])[0])
df_pairs_neg['Title_2'] = df_pairs_neg['Issue_id_2'].apply(lambda x: list(df[df['Issue_id'] == x]['Desc_list'])[0])
df_pairs_neg['label'] = 0
df_pairs_neg.head()

,Issue_id_1,Issue_id_2,Resolution,Title_1,Title_2,label
0,TST-170,SPARK-290,Not A Problem,"[increas, svn, limit, bin, artifact, 300mb]","[us, spark_master_ip, set, start, slav, sh]",0
1,TST-171,SPARK-290,Invalid,"[javapairrd, support, subtractbykey]","[us, spark_master_ip, set, start, slav, sh]",0
2,TST-168,SPARK-290,Fixed,"[spark, shel, wil, fail, start, spark, deploy,...","[us, spark_master_ip, set, start, slav, sh]",0
3,TST-169,SPARK-290,Unresolved,"[spark, stat, pack, support, common, stat, est...","[us, spark_master_ip, set, start, slav, sh]",0
4,SPARK-290,SPARK-291,Fixed,"[us, spark_master_ip, set, start, slav, sh]","[remov, cred, lin, build]",0


In [12]:
print df_pairs_neg.shape
print df_pairs_pos.shape

(1588, 6)
(1587, 3)


In [13]:
df_pairs_pos['Title_1'] = df_pairs_pos['Issue_id_1'].apply(lambda x: list(df[df['Issue_id'] == x]['Desc_list'])[0])
df_pairs_pos['Title_2'] = df_pairs_pos['Issue_id_2'].apply(lambda x: list(df[df['Issue_id'] == x]['Desc_list'])[0])
df_pairs_pos['label'] = 1
df_pairs_pos.head()

,Issue_id_1,Issue_id_2,Resolution,Title_1,Title_2,label
0,SPARK-533,SPARK-736,Fixed,"[kil, task, spark, request, com]","[kil, task, spark]",1
1,SPARK-545,SPARK-983,Duplicate,"[support, extern, sort]","[support, extern, sort, rdd, sortbykey]",1
2,SPARK-594,SPARK-612,Fixed,"[upd, exampl, pass, jar, build, sparkcontext, ...","[upd, exampl, pass, jar, fil, sparkcontext, ma...",1
3,SPARK-620,SPARK-671,Duplicate,"[default, spark_mem, high]","[spark, run, mem, fork/exec, affect, pip, python]",1
4,SPARK-636,SPARK-650,Unresolved,"[ad, mech, run, system, management/configurati...","[ad, setup, hook, ap, run, init, cod, execut]",1


In [29]:
ratios = [0.6, 0.3, 0.1]
train_set = pd.concat([df_pairs_neg.iloc[range(int(ratios[0]*len(df_pairs_neg)))],df_pairs_pos.iloc[range(int(ratios[0]*len(df_pairs_pos)))]])
test_set = pd.concat([df_pairs_neg.iloc[range(int(ratios[0]*len(df_pairs_neg)), int((ratios[1] + ratios[0])*len(df_pairs_neg)))],df_pairs_pos.iloc[range(int(ratios[0]*len(df_pairs_pos)), int((ratios[1] + ratios[0])*len(df_pairs_pos)))]])
vali_set = pd.concat([df_pairs_neg.iloc[range(int((ratios[1] + ratios[0])*len(df_pairs_neg)), len(df_pairs_neg))],df_pairs_pos.iloc[range(int((ratios[1] + ratios[0])*len(df_pairs_pos)), len(df_pairs_pos))]])


In [30]:
print train_set.shape
print test_set.shape
print vali_set.shape

(1904, 6)
(953, 6)
(318, 6)


In [31]:
print df_pairs_neg.shape
print df_pairs_pos.shape

(1588, 6)
(1587, 6)


In [3]:
df = pd.read_csv(open('../datas/spark.csv', 'rU'))
df['Duplicate_null'] = df['Duplicated_issue'].apply(lambda x : pd.isnull(x))
df_data = df[df['Duplicate_null'] == False]
prep = Preprocess()
df_data['Desc_list'] = df_data['Title'].apply(lambda x : prep.stem_and_stop_removal(x))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
df_field = df_data[['Issue_id', 'Desc_list', 'Duplicated_issue', 'Resolution']]
df_field.shape

(2806, 4)

In [36]:
df_field.head(100)

,Issue_id,Desc_list,Duplicated_issue,Resolution,dup_list
246,SPARK-533,"[kil, task, spark, -, request, com]",SPARK-736,Fixed,[SPARK-736]
258,SPARK-545,"[support, extern, sort]",SPARK-983,Duplicate,[SPARK-983]
307,SPARK-594,"[upd, exampl, pass, jar, build, sparkcontext, ...",SPARK-612,Fixed,[SPARK-612]
325,SPARK-612,"[upd, exampl, pass, jar, fil, sparkcontext, ma...",SPARK-594,Duplicate,[SPARK-594]
333,SPARK-620,"[default, spark_mem, high]",SPARK-671,Duplicate,[SPARK-671]
349,SPARK-636,"[ad, mech, run, system, management/configurati...",SPARK-650;SPARK-3513,Unresolved,"[SPARK-650, SPARK-3513]"
363,SPARK-650,"[ad, ``, setup, hook, '', ap, run, init, cod, ...",SPARK-636,Unresolved,[SPARK-636]
368,SPARK-655,"[impl, co-partitioning, aw, join, pyspark]",SPARK-5785,Duplicate,[SPARK-5785]
384,SPARK-671,"[spark, run, mem, fork/exec, affect, pip, python]",SPARK-620,Fixed,[SPARK-620]
385,SPARK-672,"[execut, get, stuck, ``, zomby, '', stat, aft,...",SPARK-1989,Duplicate,[SPARK-1989]


In [32]:
df_field['dup_list'] = df_field['Duplicated_issue'].apply(lambda x: x.split(';'))
df_field.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Issue_id,Desc_list,Duplicated_issue,Resolution,dup_list
246,SPARK-533,"[kil, task, spark, -, request, com]",SPARK-736,Fixed,[SPARK-736]
258,SPARK-545,"[support, extern, sort]",SPARK-983,Duplicate,[SPARK-983]
307,SPARK-594,"[upd, exampl, pass, jar, build, sparkcontext, ...",SPARK-612,Fixed,[SPARK-612]
325,SPARK-612,"[upd, exampl, pass, jar, fil, sparkcontext, ma...",SPARK-594,Duplicate,[SPARK-594]
333,SPARK-620,"[default, spark_mem, high]",SPARK-671,Duplicate,[SPARK-671]


In [45]:
Dup_list = []
for i,r in df_field.iterrows():
    for dup in r['dup_list']:
        if int(r['Issue_id'].split('-')[1]) < int(dup.split('-')[1]):
            Dup_list.append([r['Issue_id'], dup, r['Resolution']])
print len(Dup_list)

1590


In [46]:
df_pairs = pd.DataFrame(Dup_list, columns = ['Issue_id_1', 'Issue_id_2', 'Resolution'])
df_pairs.head()

,Issue_id_1,Issue_id_2,Resolution
0,SPARK-533,SPARK-736,Fixed
1,SPARK-545,SPARK-983,Duplicate
2,SPARK-594,SPARK-612,Fixed
3,SPARK-620,SPARK-671,Duplicate
4,SPARK-636,SPARK-650,Unresolved


In [52]:
i = 1
df_pairs.ix[i+1]

Issue_id_1    SPARK-594
Issue_id_2    SPARK-612
Resolution        Fixed
Name: 2, dtype: object

In [18]:
## 构建pair
# df_field[df_field['Issue_id'] == 'SPARK-533']['Desc_list']
df_field['Dup_desc_list'] = df_field['dup_list'].apply(lambda x: list(df_field[df_field['Issue_id'] == x]['Desc_list']))

IndexError: list index out of range